In [6]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [1]:
from weather_api import *

In [2]:
df = get_data("pszczyna")
df
#Jak widać kolumny dają nam za mało informacji, trzeba obgadac które dane wyciagnac

,Nazwa stacji,Rok,Miesiąc,Dzień,Średnia dobowa temperatura [°C],Status pomiaru TEMP,Status pomiaru WLGS,Średnia dobowa prędkość wiatru [m/s],Status pomiaru FWS,Średnie dobowe zachmurzenie ogólne [oktanty],Status pomiaru NOS
0,249180010,PSZCZYNA,2019,1,-1.671958,-0.877129,NaN,NaN,NaN,NaN,1.348303
1,249180010,PSZCZYNA,2019,1,-1.558356,-1.176911,NaN,NaN,NaN,NaN,1.348303
2,249180010,PSZCZYNA,2019,1,-1.444753,-1.694718,NaN,NaN,NaN,NaN,1.348303
3,249180010,PSZCZYNA,2019,1,-1.331150,-1.708344,NaN,NaN,NaN,NaN,1.348303
4,249180010,PSZCZYNA,2019,1,-1.217548,-1.313176,NaN,NaN,NaN,NaN,1.348303
...,...,...,...,...,...,...,...,...,...,...,...
512,249180010,PSZCZYNA,2019,12,1.281710,-0.972514,NaN,NaN,NaN,NaN,1.348303
513,249180010,PSZCZYNA,2019,12,1.395313,-1.245044,NaN,NaN,NaN,NaN,1.348303
514,249180010,PSZCZYNA,2019,12,1.508915,-1.299550,NaN,NaN,NaN,NaN,1.348303
515,249180010,PSZCZYNA,2019,12,1.622518,-1.326803,NaN,NaN,NaN,NaN,0.660695


In [2]:
__all__ = ["get_data"]

DATA_FOLDER = "./data"

def read_column_names(file):
    with open(file, 'r') as f:
        content = f.readlines()
    
    return [x.strip() for x in content]

def delete_mostly_nan_columns(df):
    column_names = list()
    all_entries = len(df.index)
    nan_values = df.isnull().sum().to_numpy()

    for i in range(len(nan_values)):
        if (nan_values[i]/all_entries)>=0.3:
            column_names.append(df.columns[i])

    for name in column_names:
        if name in df.columns:
            del df[name]

def normalize_particular_columns(df, column_names):
    for col_name in column_names:
        df.iloc[:,col_name]=(df.iloc[:,col_name]-df.iloc[:,col_name].mean())/df.iloc[:,col_name].std()


In [7]:
columns_names = read_column_names('./data/head.txt')
df1 = pd.read_csv('./data/01_2020.csv', names=columns_names[1:], index_col=False, delimiter = ';')
df2 = pd.read_csv('./data/02_2020.csv', names=columns_names[1:], index_col=False, delimiter = ';')
df3 = pd.read_csv('./data/03_2020.csv', names=columns_names[1:], index_col=False, delimiter = ';')
df4 = pd.read_csv('./data/04_2020.csv', names=columns_names[1:], index_col=False, delimiter = ';')
df5 = pd.read_csv('./data/05_2020.csv', names=columns_names[1:], index_col=False, delimiter = ';')
df6 = pd.read_csv('./data/06_2020.csv', names=columns_names[1:], index_col=False, delimiter = ';')
delete_mostly_nan_columns(df1)
delete_mostly_nan_columns(df2)
delete_mostly_nan_columns(df3)
delete_mostly_nan_columns(df4)
delete_mostly_nan_columns(df5)
delete_mostly_nan_columns(df6)

data = [df1,df2,df3,df4,df5,df6]
result = pd.concat(data)

In [8]:
result.head()

,Kod stacji,Nazwa stacji,Rok,MiesiÄ…c,DzieĹ„,Ĺšrednie dobowe zachmurzenie ogĂłlne [oktanty],Ĺšrednia dobowa prÄ™dkoĹ›Ä‡ wiatru [m/s],Ĺšrednia dobowa temperatura [Â°C],Ĺšrednia dobowe ciĹ›nienie pary wodnej [hPa],Ĺšrednia dobowa wilgotnoĹ›Ä‡ wzglÄ™dna [%],Ĺšrednia dobowe ciĹ›nienie na poziomie stacji [hPa],Ĺšrednie dobowe ciĹ›nienie na pozimie morza [hPa],Status pomiaru PPPM,Suma opadu dzieĹ„ [mm],Suma opadu noc [mm]
0,349190625,ZAKOPANE,2020,1,1,4.3,2.0,-1.7,4.0,73.1,929.5,0.0,8,0.0,0.0
1,349190625,ZAKOPANE,2020,1,2,4.4,1.3,-1.5,3.1,56.8,928.8,0.0,8,0.0,0.0
2,349190625,ZAKOPANE,2020,1,3,2.5,1.4,-1.1,3.8,68.8,922.4,0.0,8,0.0,0.0
3,349190625,ZAKOPANE,2020,1,4,7.6,3.6,0.8,5.4,83.6,917.5,0.0,8,1.9,6.5
4,349190625,ZAKOPANE,2020,1,5,7.9,2.8,-4.1,3.8,83.6,924.4,0.0,8,3.4,0.0


In [9]:
value = []
data = result.iloc[:,[13]]
data = data.to_numpy()
for row in data:
    if row == 0:
        value.append(0)
    elif row > 0:
        value.append(1)
result['deszcz w dzień']= value

value1 = []
data1 = result.iloc[:,[14]]
data1 = data1.to_numpy()
for row1 in data1:
    if row1 == 0:
        value1.append(0)
    elif row1 > 0:
        value1.append(1)
result['deszcz w nocy']= value

In [10]:
result

,Kod stacji,Nazwa stacji,Rok,MiesiÄ…c,DzieĹ„,Ĺšrednie dobowe zachmurzenie ogĂłlne [oktanty],Ĺšrednia dobowa prÄ™dkoĹ›Ä‡ wiatru [m/s],Ĺšrednia dobowa temperatura [Â°C],Ĺšrednia dobowe ciĹ›nienie pary wodnej [hPa],Ĺšrednia dobowa wilgotnoĹ›Ä‡ wzglÄ™dna [%],Ĺšrednia dobowe ciĹ›nienie na poziomie stacji [hPa],Ĺšrednie dobowe ciĹ›nienie na pozimie morza [hPa],Status pomiaru PPPM,Suma opadu dzieĹ„ [mm],Suma opadu noc [mm],deszcz w dzień,deszcz w nocy
0,349190625,ZAKOPANE,2020,1,1,4.3,2.0,-1.7,4.0,73.1,929.5,0.0,8,0.0,0.0,0,0
1,349190625,ZAKOPANE,2020,1,2,4.4,1.3,-1.5,3.1,56.8,928.8,0.0,8,0.0,0.0,0,0
2,349190625,ZAKOPANE,2020,1,3,2.5,1.4,-1.1,3.8,68.8,922.4,0.0,8,0.0,0.0,0,0
3,349190625,ZAKOPANE,2020,1,4,7.6,3.6,0.8,5.4,83.6,917.5,0.0,8,1.9,6.5,1,1
4,349190625,ZAKOPANE,2020,1,5,7.9,2.8,-4.1,3.8,83.6,924.4,0.0,8,3.4,0.0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,349190625,ZAKOPANE,2020,6,26,6.4,1.1,17.7,14.4,74.0,921.9,0.0,8,1.1,1.2,1,1
26,349190625,ZAKOPANE,2020,6,27,5.9,2.3,21.2,14.1,56.9,918.7,0.0,8,0.0,1.3,0,0
27,349190625,ZAKOPANE,2020,6,28,3.6,1.5,19.6,15.8,71.9,919.0,0.0,8,0.0,0.0,0,0
28,349190625,ZAKOPANE,2020,6,29,4.9,1.5,17.7,14.8,76.5,915.3,0.0,8,3.6,0.3,1,1


In [11]:
columns_to_normalize = [[5],[6],[8],[9],[10]]
normalize_particular_columns(result, columns_to_normalize)

In [12]:
result = result.drop(result.columns[11],axis = 1)
result.head()

,Kod stacji,Nazwa stacji,Rok,MiesiÄ…c,DzieĹ„,Ĺšrednie dobowe zachmurzenie ogĂłlne [oktanty],Ĺšrednia dobowa prÄ™dkoĹ›Ä‡ wiatru [m/s],Ĺšrednia dobowa temperatura [Â°C],Ĺšrednia dobowe ciĹ›nienie pary wodnej [hPa],Ĺšrednia dobowa wilgotnoĹ›Ä‡ wzglÄ™dna [%],Ĺšrednia dobowe ciĹ›nienie na poziomie stacji [hPa],Status pomiaru PPPM,Suma opadu dzieĹ„ [mm],Suma opadu noc [mm],deszcz w dzień,deszcz w nocy
0,349190625,ZAKOPANE,2020,1,1,-0.395931,0.354113,-1.7,-0.734239,0.140533,1.524346,8,0.0,0.0,0,0
1,349190625,ZAKOPANE,2020,1,2,-0.352936,-0.553613,-1.5,-0.988991,-0.995140,1.432098,8,0.0,0.0,0,0
2,349190625,ZAKOPANE,2020,1,3,-1.169839,-0.423938,-1.1,-0.790851,-0.159062,0.588682,8,0.0,0.0,0,0
3,349190625,ZAKOPANE,2020,1,4,1.022900,2.428913,0.8,-0.337959,0.872102,-0.057058,8,1.9,6.5,1,1
4,349190625,ZAKOPANE,2020,1,5,1.151885,1.391513,-4.1,-0.790851,0.872102,0.852249,8,3.4,0.0,1,1


In [15]:
X = result[[result.columns[5],result.columns[6],result.columns[7],result.columns[8],result.columns[9],result.columns[10]]]
y = result[[result.columns[14]]]

In [18]:
Xtrain, Xtest, ytrain, ytest  = train_test_split(X,y,test_size=0.2,random_state=75)

Model SVN 

In [21]:
clf= svm.SVC(kernel='linear', C=1.0)
clf.fit(Xtrain,ytrain)

C:\Users\surface2\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [22]:
ypred = clf.predict(Xtest)
clf_score = clf.score(Xtest,ytest)

In [23]:
clf_score

0.8378378378378378

In [25]:
confusion_matrix(ytest,ypred)

array([[16,  3],
       [ 3, 15]], dtype=int64)

Model logistic regresion

In [28]:
logistic_regression= LogisticRegression()
logistic_regression.fit(Xtrain,ytrain)
y_pred_r=logistic_regression.predict(Xtest)
    
accur = accuracy_score(ytest, y_pred_r)

C:\Users\surface2\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\surface2\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [29]:
accur

0.8108108108108109

In [31]:
confusion_matrix(ytest,y_pred_r)

array([[16,  3],
       [ 4, 14]], dtype=int64)

Model KNN

In [39]:
knn = KNeighborsClassifier()

In [40]:
knn.fit(Xtrain, ytrain)
y_pred_knn = knn.predict(Xtest)
accurKNN = accuracy_score(ytest, y_pred_knn)

C:\Users\surface2\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """Entry point for launching an IPython kernel.


In [41]:
accurKNN

0.8108108108108109

In [42]:
confusion_matrix(ytest,y_pred_knn)

array([[16,  3],
       [ 4, 14]], dtype=int64)